# match_geos

```
!pip3 install python-Levenshtein
```

In [33]:
import os
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [34]:
print(os.getcwd())
REF_DATA = '../data/geo_files/bbs_geos/geo.xlsx'
INPUT_DATA = '../data/dgfp/dgfp_geo.csv'

/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/utils


In [35]:
def process_zilas_upazilas(path):
    tmp = pd.read_excel(path, header=1)
    tmp = tmp[['Division Code','Division name', 'District Code','District Name','Upazilla code','Upazilla']]
    tmp.columns = ['DivisionCode','DivisionName', 'DistrictCode','DistrictName','UpazillaCode','UpazillaName']
    tmp['DistrictName'] = tmp['DistrictName'].str.replace(' Zila', '')
    tmp['UpazillaName'] = tmp['UpazillaName'].str.replace(' Upazila', '')
    tmp = tmp.drop_duplicates()
    return tmp, list(tmp.columns)

def get_input_geos(path):
    tmp = pd.read_csv(path)
    display(tmp.head())
    return tmp

def partial_match(ref_df, ref_match, input_df, input_match):
    out = pd.DataFrame()
    for key, code in enumerate(input_df[input_match]):
        code_match = {}
        code_match['FuzzRatio'] = [fuzz.ratio(ref_code, code)  for ref_code in ref_df[ref_match]]
        code_match['FuzzRatioDivision'] = [fuzz.ratio(ref_df.iloc[k,1], input_data.iloc[key,5])  for k,ref_code in enumerate(ref_df[ref_match])]
        code_match['InputDivision'] = input_data.iloc[key,5]
        code_match['Geo'] = code
        code_match['DivisionCode'] = [value[0] for value in ref_df.values]
        code_match['DivisionName'] = [value[1]  for value in ref_df.values]
        code_match['DistrictCode'] = [value[2]  for value in ref_df.values]
        code_match['DistrictName'] = [value[3]  for value in ref_df.values]
        code_match['UpazillaCode'] = [value[4]  for value in ref_df.values]
        code_match['UpazillaName'] = [value[5]  for value in ref_df.values]
        code_match = pd.DataFrame.from_dict(code_match)
        code_match['MeanFuzzRation'] = np.round((code_match['FuzzRatio'] + code_match['FuzzRatioDivision'])/2,2)
        out = out.append(code_match.sort_values('MeanFuzzRation', ascending=False).iloc[0,:])
    return out
   

# test

In [36]:
bbs_geos, geo_levels = process_zilas_upazilas(REF_DATA)
input_data = get_input_geos(path=INPUT_DATA)
old_new = {'01': 'Rajshahi',
           '02': 'Khulna',
           '03': 'Barisal',
           '04': 'Dhaka',
           '05': 'Chittagong',
           '06': 'Sylhet',
           '07':'Rangpur', 
           '08': 'Mymensingh',
          }
input_data['DivisionName'] = input_data['Division'].astype('category')
input_data['DivisionName'] = input_data['DivisionName'].cat.rename_categories(old_new)
display(input_data.head())

,Unnamed: 0,thana,District,Division,program
0,0,Boda,103,07,thana_process
1,1,Panchgarh,103,07,thana_process
2,2,Tetulia,103,07,thana_process
3,3,Atowari,103,07,thana_process
4,4,Debiganj,103,07,thana_process


,Unnamed: 0,thana,District,Division,program,DivisionName
0,0,Boda,103,07,thana_process,Rangpur
1,1,Panchgarh,103,07,thana_process,Rangpur
2,2,Tetulia,103,07,thana_process,Rangpur
3,3,Atowari,103,07,thana_process,Rangpur
4,4,Debiganj,103,07,thana_process,Rangpur


In [37]:
bbs_geos.head()

,DivisionCode,DivisionName,DistrictCode,DistrictName,UpazillaCode,UpazillaName
0,40,Khulna,1,Bagerhat,8,Bagerhat Sadar
9,40,Khulna,1,Bagerhat,58,Mongla
17,40,Khulna,1,Bagerhat,60,Morrelganj
26,20,Chittagong,3,Bandarban,14,Bandarban Sadar
35,20,Chittagong,3,Bandarban,51,Lama


In [38]:
test = partial_match(ref_df=bbs_geos, ref_match='UpazillaName', input_df=input_data, input_match='thana')

In [41]:
test.to_csv("../output/geo_files/fuzz_geo_matched.csv", index=False, index_label=False)